SeHGNN Implementation
This implementation includes:

1.Simplified Neighbor Aggregation → Precomputes neighbor aggregation.

2.Multi-layer Feature Projection → Uses MLPs to project features.

3.Transformer-based Semantic Fusion → Uses a Transformer module to combine metapath information.


In [2]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 108.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 6

In [3]:
!pip install torch-geometric


  Using cached torch_geometric-2.6.1-py3-none-any.whl.metadata (63 kB)
Using cached torch_geometric-2.6.1-py3-none-any.whl (1.1 MB)


In [4]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html


Looking in links: https://data.pyg.org/whl/torch-2.0.0+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 51.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 65.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 886.5/886.5 kB 42.9 MB/s eta 0:00:00


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import softmax
import math

class NeighborAggregation(nn.Module):
    def __init__(self, metapaths, in_channels):
        super().__init__()
        self.metapaths = metapaths
        self.in_channels = in_channels

    def forward(self, x, edge_index_dict):
        """Precomputes neighbor aggregation for each metapath"""
        aggregated_features = {}
        for metapath in self.metapaths:
            edge_index = edge_index_dict[metapath]
            row, col = edge_index
            agg_feature = torch.zeros_like(x)
            agg_feature.index_add_(0, row, x[col])
            agg_feature /= torch.clamp(torch.bincount(row, minlength=x.size(0)).unsqueeze(1), min=1)
            aggregated_features[metapath] = agg_feature
        return aggregated_features

# ---- Step 2: Multi-layer Feature Projection (MLP) ----
class FeatureProjection(nn.Module):
    def __init__(self, in_channels, out_channels, metapaths):
        super().__init__()
        self.metapaths = metapaths
        self.mlps = nn.ModuleDict({
            metapath: nn.Sequential(
                nn.Linear(in_channels, out_channels),
                nn.ReLU(),
                nn.Linear(out_channels, out_channels)
            ) for metapath in metapaths
        })

    def forward(self, aggregated_features):
        """Projects features to a uniform space using MLPs"""
        projected_features = {mp: self.mlps[mp](feat) for mp, feat in aggregated_features.items()}
        return projected_features

# ---- Step 3: Transformer-based Semantic Fusion ----
class TransformerSemanticFusion(nn.Module):
    def __init__(self, out_channels, num_metapaths, num_heads=1):
        super().__init__()
        self.num_heads = num_heads
        self.out_channels = out_channels

        # Transformer parameters
        self.query = nn.Linear(out_channels, out_channels)
        self.key = nn.Linear(out_channels, out_channels)
        self.value = nn.Linear(out_channels, out_channels)
        self.softmax = nn.Softmax(dim=-1)
        self.beta = nn.Parameter(torch.ones(1))

    def forward(self, projected_features):
        """Uses Transformer attention to merge metapath-based embeddings"""
        metapath_list = list(projected_features.keys())
        feature_matrix = torch.stack([projected_features[mp] for mp in metapath_list], dim=1)

        Q = self.query(feature_matrix)
        K = self.key(feature_matrix)
        V = self.value(feature_matrix)

        attention_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.out_channels)
        attention_weights = self.softmax(attention_scores)
        fusion = self.beta * torch.matmul(attention_weights, V) + feature_matrix
        return fusion.mean(dim=1)

# ---- Putting it all together ----
class SeHGNN(nn.Module):
    def __init__(self, in_channels, out_channels, metapaths):
        super().__init__()
        self.aggregation = NeighborAggregation(metapaths, in_channels)
        self.projection = FeatureProjection(in_channels, out_channels, metapaths)
        self.fusion = TransformerSemanticFusion(out_channels, len(metapaths))

    def forward(self, x, edge_index_dict):
        aggregated_features = self.aggregation(x, edge_index_dict)
        projected_features = self.projection(aggregated_features)
        node_embeddings = self.fusion(projected_features)
        return node_embeddings

if __name__ == "__main__":
    num_nodes = 100
    in_channels = 64
    out_channels = 32
    metapaths = ["PA", "PS", "PAP", "PSP"]

    # Random input features
    x = torch.randn((num_nodes, in_channels))

    edge_index_dict = {
        "PA": torch.randint(0, num_nodes, (2, 300)),
        "PS": torch.randint(0, num_nodes, (2, 300)),
        "PAP": torch.randint(0, num_nodes, (2, 300)),
        "PSP": torch.randint(0, num_nodes, (2, 300)),
    }

    # Initialize and run SeHGNN
    model = SeHGNN(in_channels, out_channels, metapaths)
    node_embeddings = model(x, edge_index_dict)
    print("Node Embeddings Shape:", node_embeddings.shape)


/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_scatter/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_cluster/_version_cuda.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/usr/local/lib/python3.11/dist-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /usr/local/lib/python3.11/dist-packages/torch_spline_conv/_version_cuda.so: undefined symbol: _ZN3c1017Registe

Node Embeddings Shape: torch.Size([100, 32])
